In [40]:
from lxml import etree
book=etree.parse('src/book.6.xml',etree.XMLParser(remove_blank_text=True)).getroot()
# re-indentation: no need to do anything
if 0:
    # verse at the beginning of the paragraph, first line as running text
    beginVerse=['IV.66','XVII.161','XVII.210','XVII.212','XVII.218','XVII.230','XVII.237','V.27',]
    for a in beginVerse:
        pp=book.findall('.//p[@anchor="'+a+'"]')
        assert len(pp)==1
        p=pp[0]
        print(len(p),p[0].tag)
        assert (len(p)==1 and p[0].tag=='span') or (len(p)==2 and p[0].tag=='printed_page' and p[1].tag=='span')
        v=p.getnext()
        assert v.tag=='verse'
        for _ in reversed(p): v.insert(0,_)
        v.attrib['y']=p.attrib['y']
        
if 0:
    # last line of paragraph broken off
    endParaFragment=['II.38','II.42','II.55','II.70','II.74','III.73','III.122','III.127',
                     'IV.87',
                     'XVII.232','XVII.238','XVII.248','XVII.260','XVII.272','XVII.281',
                     'XVII.291','XVIII.7','XX.29','XX.42']
    for anchor in endParaFragment:
        pp=book.findall('.//p[@anchor="'+anchor+'"]')
        assert len(pp)==1
        para=pp[0]
        # print(anchor,para,para.sourceline)
        paraAfter=para.getnext()
        while not 'anchor' in paraAfter.attrib: paraAfter=paraAfter.getnext()
        para1=paraAfter.getprevious()
        para0=para1.getprevious()
        #print(paraAfter.attrib['anchor'])
        #print(paraLast[-1].tag)
        print(anchor,len(para1),[e.tag for e in para1])
        print('  ',para0[-1].text)
        print('  ',para1[0].text)
        assert(para0[-1].tag==para1[0].tag)
        para0[-1].text+=' '+para1[0].text
        para1.remove(para1[0])
        for e in para1: para0.append(e)
        # print('   ',len(para0),[e.tag for e in para0])
        # print(f'{para[-1].tag=}')

if 0:
    for p in book.findall('.//p'):
        if len(p)>1 or len(p)==0: continue
        e0=p[0]
        if e0.tag not in ['em','span']: continue
        assert e0.text is not None
        if len(e0.text)>100: continue
        #print(e0.text)
        if e0.text.startswith('[') and e0.text.endswith(']') and e0.text[-2]!=':': print(e0.text)
        elif 'family' in e0.attrib: print(e0.attrib['family'],e0.text)
        

if 0:        
    for p in book.findall('.//p'):
        if len(p)!=1: continue
        if p[0].tag!='em': continue
        if not (p[0].text.startswith('[') and p[0].text.endswith(']')): continue
        if 'title' not in p.attrib: p.attrib['title']='1'
if 1:
    for p in book.findall('.//p[@title]'):
        del p.attrib['title']
        p.attrib['editorial_title']="1"
        # print(p.attrib['title'])
if 1:
    def _E(tag,text=None,**kw):
        ret=etree.Element(tag,**kw)
        ret.text=text
        return ret
        
    import re
    for span in book.findall('.//span'):
        if (pp:=span.getparent().getparent()).tag=='footnote': continue
        if '§' not in span.text: continue
        ch=span.getparent()
        while ch.tag!='struct-2-chapter':  ch=ch.getparent()
        chap=ch.find('.//heading-2-chapter').get('toc_num',None)
        noMatch=True
        while m:=re.search('(\[|\(|—|, )(|cf. |see |See )(?P<para>§(?P<num>[0-9]+)(|f.|ff.))(\]|\)|;|,| below| above| and)',span.text):
            noMatch=False
            span.addprevious(_E('span',text=span.text[:m.start('para')]))
            span.addprevious(_E('ref',text=m.group('para'),type="vism",target=f'{chap}.{m.group("num")}'))
            print(chap,span.sourceline,m.group('para'))  
            span.text=span.text[m.end('para'):]
        if noMatch:
            while m:=re.search('(?P<head>§§|§)(?P<p1>[0-9]+)(?P<delim>–|,)(?P<p2>[0-9]+)\)',span.text):
                noMatch=False
                print(chap,m.group('head'),m.group('p1'),m.group('delim'),m.group('p2'))
                span.addprevious(_E('span',text=span.text[:m.start('head')]))
                span.addprevious(_E('ref',text=m.group('head')+m.group('p1'),type="vism",target=f'{chap}.{m.group("p1")}'))
                span.addprevious(_E('span',text=m.group('delim')))
                span.addprevious(_E('ref',text=m.group('p2'),type="vism",target=f'{chap}.{m.group("p2")}'))
                span.text=span.text[m.end('p2'):]
        if noMatch:
            print(span.sourceline,span.text)
                            
            # print(m)
            
        # if not m: print(span.text)
            
            # print(m)
            #ix=span.text.index('§')
            #print(span.text[max(0,ix-20):ix+20],pp.tag)
        

open('src/book.6a.xml','w').write(etree.tostring(book,encoding='unicode',pretty_print=True))

3605280

In [57]:
from lxml import etree
ix=etree.parse('src/index.xml',etree.XMLParser(remove_blank_text=True)).getroot()

if 0:

    def splitEntry(entry,splitter):
        assert splitter.tag=='Xspan'
        ixs=entry.index(splitter)
        ent2=etree.Element('entry',page_id=entry.attrib['page_id'],title=splitter.text.strip())
        ent2[:]=entry[ixs+1:]
        entry.remove(splitter)
        entry.addnext(ent2)
        return ent2

    for e in ix.findall('.//Xspan'):
        assert (ent:=e.getparent()).tag=='entry'
        if e.text.endswith('('):
            assert (n:=e.getnext()).tag=='em'
            assert (nn:=e.getnext().getnext()).tag=='span'
            e.text=e.text[:-1]
            ent2=splitEntry(ent,e)
            ent2.insert(0,sp:=etree.Element('span'))
            sp.text='('
        else:
            assert (n:=e.getnext()).tag=='ref'
            splitEntry(ent,e)

        
for ent in ix.findall('.//entry'):
    if len(ent)==0: print(ent.sourceline)
    if len(ent)==1: continue
    span=ent[0]
    ref=ent[1]
    if span.tag!='span' or ref.tag!='ref': continue
    tx=span.text
    if tx.startswith(','):
        print(ent.sourceline,ent.attrib['title'],span.text)
        ent.attrib['title']+=tx.rstrip()
        ent.remove(span)
        #span.text='('
        # print(span.text)
    
    # assert e.tag=='entry'
    # print(e)
    

open('src/index.2.xml','w').write(etree.tostring(ix,encoding='unicode',pretty_print=True))

34 Abhaya Thera , Tipiṭaka Cūḷa 
47 Abhaya Thera , Dīghabhāṇaka 
54 Abhaya Thera , Pīṭha 
656 Anāthapiṇḍika , Cūḷa 
2340 Chaddanta , Lake 
2462 Citta , householder 
4056 Datta Thera , Mahā 
4786 Dhammarakkhita Thera , Mahā 
7123 Godatta Thera , Abhidhammika 
7375 Gutta Thera , Mahā Rohaṇa 
7387 Haṃsapātana , Lake 
8903 Kaccāna Thera , Mahā 
9118 Kaṇṇamuṇḍaka , Lake 
9126 Kappina Thera , Mahā 
9130 Karañjiya-vihāra , Mahā 
9379 Kuṇāla , Lake 
10610 Mitta Thera , Mahā 
10633 Moggallāna Thera , Mahā 
10835 Nāga Thera , Tipiṭaka Cūḷa 
12136 Panthaka Thera , Cūḷa 
13199 Rathakāra , Lake 
13801 Revata Thera , Majjhimabhāṇaka 
14032 Rohaṇa-Gutta Thera , Mahā 
14835 Sīhapapāta , Lake 
14843 Sineru , Mount 
14887 Siva Thera , Cūḷa 
15295 Subhaddā , Cūḷa 
15924 Tissa Thera , Cūḷa-piṇḍapātika 
15931 Tissa Thera , Koṭapabbatavāsin 
15948 Tissa Thera , Mahā, Mahā-Karañjiya-vihāra-vāsin 
15953 Tissa Thera , Padhāniya, Nāgapabbatavāsin 
15958 Tissa Thera , Piṇḍapātika, Devaputtaraṭṭhavāsin 


557985